## Requirements

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot
- All people in the company have between 25 and 40 years, give them some place to go to party.
- Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
- The CEO is Vegan

In [1]:
from pymongo import MongoClient

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [ ]:
def getLocation(wendy):
    longitude = wendy['address']['coord'][0]['$numberDouble']
    latitude = wendy['address']['coord'][1]['$numberDouble']
    loc = {
        'type':'Point',
        'coordinates':[float(longitude), float(latitude)]
    }
    return loc

In [3]:
db, coll = connectCollection('companies','companies')

In [23]:
new_companies = coll.find({'founded_year': {'$gt': 2009}})

In [24]:
print(company[0]['offices'][0]['latitude'])

37.09024


In [25]:
print(company[0]['offices'][0]['longitude'])

-95.712891
